## 1. 提取物料分类信息、小类名称-编码字典

In [1]:
# 创建.xlsx文档
from openpyxl import Workbook, load_workbook
import os
import glob
import re
import numpy as np
import pandas as pd

from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
## 提取分类类别
Y201_ruls_path = 'base_template/Y201 物料分类及编码规则 R21.xlsm'
Y201_ruls_wb = load_workbook(Y201_ruls_path)
Y201_ruls_ws = Y201_ruls_wb['物料分类表 R21']

# C: 大类名称 电气……
# E：中类名称 传感器……
# F: 次中类名称 连续物位……
# G：小类名称 雷达……
# H：小类代号 A300……
C, D, E, F, G, H = Y201_ruls_ws['C:H']
primary_class = {'电气':set()}  #大类：中类
medium_class = {}             #中类：次中类
detailed_class = {}            #次中类：小类
class_code = {}

for c, e, f, g, h  in zip(C, E, F, G, H):
    if c.value == '电气':
        primary_class['电气'].add(e.value)
        # 存储中类：次中类
        try:
            medium_class[e.value].add(f.value)
        except:
            medium_class[e.value] = set()
            medium_class[e.value].add(f.value)
        # 存储 次中类：小类
        try:
            detailed_class[f.value].add(g.value)
        except:
            detailed_class[f.value] = set()
            detailed_class[f.value].add(g.value)
        # 存储 小类名称：小类编码
        class_code[g.value] = h.value
        
print(medium_class.keys())
print(medium_class['传感器'])
print(detailed_class['连续物位测量'])

dict_keys(['电气类', '工控系统', '电气安装材料', '传感器', '自控阀门'])
{'流量测量', '连续物位测量', '传感器安装附件', '火花检测', '分析仪表', '粉尘检测', '物位开关', '振动测量', '密度检测', '称重装置', '可燃/有毒检测', '压力测量', '温/湿度测量', '其它类型的传感器', '位置检测'}
{'双法兰液位变送器', '连杆浮球液位传感器', '其它形式连续物位测量', '导播雷达物位传感器', '雷达物位传感器', '磁翻板液位计', '角度变送器', '超声波物位传感器', '投入式压力变送器', '静压式液位变送器'}


In [3]:
# 提取品牌库信息
brands_path = 'base_template/BrandsList.txt' # 最终品牌库的路径
brands_file = open(brands_path, 'r')
brs = brands_file.readlines()
brands = {}

for br in brs:
    ls = re.split("\s", br)
    if ls[0] not in brands.keys() \
    and ls[1] not in brands.values():
        brands[ls[0]] = ls[1]
    else:
        print('品牌库异常：', ls[0], ls[1], '出现重复项')
        

## 创建物料文件夹

In [4]:
class_job = '传感器'
for m_class in medium_class[class_job]:
    for d_class in detailed_class[m_class]:
        m_class = m_class.replace("/", '')
        d_class = d_class.replace("/", '')
        path = class_job + '/' + m_class + '/' + d_class
        
        if not os.path.exists(path):
            os.makedirs(path)
            print('已创建路径：', path)


## 提取小类库文件中的物料信息

In [5]:
def get_data(file_path):
    if file_path[-5:] == '.xlsx':
        wb = load_workbook(file_path, 'r')
        ws = wb.active
        data =  np.array(list(ws.values))
        wb.close
        return  data
    elif file_path[-4:] == '.txt':
        txt = open(file_path, 'r')
        txt = txt.readlines()
        data = []
        for row in txt:    
            d = row.replace("\n",'')
            data.append(re.split("\t", d))
        return np.array(data)
        
    else:
        raise ValueError

In [6]:
def get_translation(trans_path):
    ## 提取翻译库数据
    trans_file = open(trans_path, 'r')
    trans = trans_file.readlines()
    translation = {}
    for tran in trans:
        tr = re.split("\s", tran)
        if tr[0] != '' :
            translation[tr[0]] = tr[1]

    trans_file.close()
    return translation

In [7]:
def refresh_translation(attribs, trans_path):
    ### 更新翻译库，将需要导入的属性加入翻译库
    current_trans = get_translation(trans_path)
    trans_file = open(trans_path, 'a+')
    trans_file.write('\n')
    for attrib in attribs:
        if attrib not in current_trans:
            trans_file.write(attrib+'\n')
    trans_file.close()
    new_trans = get_translation(trans_path)
    for key in new_trans.keys():
        if new_trans[key] == '':
            print(key, ':翻译缺失')
            
    return new_trans

In [8]:
# wb_path = '雷达物位传感器.xlsx'
# data = get_data(wb_path)

txt_path = 'txt_read_test.txt'
data = get_data(txt_path)
data.shape

(18, 19)

## 生成PLM模板文件

In [9]:
### 生成属性

job = "雷达物位传感器"

## Common data
dataframe = pd.DataFrame(data=data[2:, 1:], columns=data[1, 1:], index=data[2:, 0])
attribs = dataframe.columns[2:]
attribs_necessity = data[0][3:]
trans_path = 'base_template/翻译库.txt'
translation = refresh_translation(attribs, trans_path)
job_code = class_code[job]

In [10]:
## 生成 02-必填属性
t02_header = '#===============电气与自动化================\n\n'
t02_header += ('#--' + job + '--' + '\n')
t02_content = job_code + '=mySupplierModelSpec'

# 遍历所有属性，添加必要属性
for attrib, attrib_necessity in zip(attribs, attribs_necessity):
    # txt读出的数值为str型，excel读出的为int型
    if attrib_necessity in [1, '1']:
        t02_content += ',my' + job_code + '_' + translation[attrib]

# 完成02模板
t02 = t02_header + t02_content

write_path = '传感器/连续物位测量/雷达物位传感器/'

t02_file = open(write_path+'02-必填属性-'+job+'.txt', 'w')
t02_file.write(t02)
t02_file.close()

In [11]:
## 生成03-属性组合
t03_header = "#====各分类组合属性集===========\n \
                #====分类码.组合属性代码+组合顺序=子属性名\n \
                #====物料简称:cassShortDescription\n \
                #====描述:myDesc\n\n"
t03_header += ('#--' + job + '--' + '\n\n')

t03_content1 = '#l表示：PLM物料名称:ERP物料名称\n' + job_code \
                + '.l0={cassShortDescription}\n\n'

t03_content2 = '#d：ERP物料名称:物料简称\n' + job_code \
                + '.d0={cassShortDescription}\n\n'

t03_content3 = "#s：ERP型号规格:供应商型号规格\n" + job_code \
                + ".s0={mySupplierModelSpec}\n\n"
t03 = t03_header + t03_content1 + t03_content2 + t03_content3

t03_file = open(write_path+'03-组合属性-'+job+'.txt', 'w')
t03_file.write(t03)
t03_file.close()


In [12]:
## 创建Excel文件
wb = Workbook()
ws_df = wb.create_sheet("定义属性")
ws_rg = wb.create_sheet("Range值")

In [13]:
## 生成定义属性工作表

ws_df_header = ['模块','中文名称','分类码','英文名字','定义属性','属性类型','翻译脚本','创建属性脚本']

attr_num = len(attribs)
ws_df_cA = ['物料属性'] * attr_num
ws_df_cB = attribs
ws_df_cC = [job_code] * attr_num
ws_df_cD = [translation[x] for x in ws_df_cB]
ws_df_cE = ['="my"&C{}&"_"&D{}'.format(i+2,i+2) for i in range(attr_num)]
ws_df_cF = ['string'] * attr_num
ws_df_cG = ['=CONCATENATE("emxFramework.Attribute.",E{}," = ",B{},)'.format(i+2,i+2) for i in range(attr_num)]
cH_string = """=CONCATENATE("#"&B{}&"
add attribute "&E{}&"
  type "&F{}&"
  description '' default ''
  property application value MyandeCentral
  property installer value cass
  property 'original name' value "&E{}&"
  property 'installed date' value 05-01-2018
  property version value 1.0;
mod prog eServiceSchemaVariableMapping.tcl add property attribute_"&E{}&" to att "&E{}&";")"""
ws_df_cH = [cH_string.format(i,i,i,i,i,i) for i in range(2, attr_num+2)]

df = pd.DataFrame(columns=ws_df_header)
df['模块'] = ws_df_cA
df['中文名称'] = ws_df_cB
df['分类码'] = ws_df_cC
df['英文名字'] = ws_df_cD
df['定义属性'] = ws_df_cE
df['属性类型'] = ws_df_cF
df['翻译脚本'] = ws_df_cG
df['创建属性脚本'] = ws_df_cH

for r in dataframe_to_rows(df, index=False, header=True):
    ws_df.append(r)
wb.save("t01.xlsx")

In [14]:
## 生成Range值工作表
ws_rg_header = ['固定属性','分类码','英文名字','定义属性','-','值的前台显示','值(不能有中文)','Range值脚本','Range值翻译']
ws_rg_cC = []
ws_rg_cF = []
range_value = []

for attrib, attrib_necessity in zip(attribs, attribs_necessity):
    ### 遍历dataframe，生成C，F列 和 属性的值
    
    # txt读出的数值为str型，excel读出的为int型
    if attrib_necessity in [1, '1']:
        for value in dataframe[attrib].unique():
            if value != '' and value != None:
                ws_rg_cC.append(translation[attrib])
                ws_rg_cF.append(value)
        if attrib != '品牌':
            for i in range(1, dataframe[attrib].nunique()+1):
                range_value.append(translation[attrib].lower() + str(i))
            
    range_value.append('')            
    ws_rg_cC.append(translation[attrib])
    ws_rg_cF.append('')

# 生成其他列，写入表格公式或值
attr_num = len(ws_rg_cF)
ws_rg_cA = ['mod attr'] * attr_num
ws_rg_cB = [job_code] * attr_num
ws_rg_cD = ['="my"&B{}&"_"&C{}'.format(i,i) for i in range(2,attr_num+2)]
ws_rg_cE = ['add range'] * attr_num

brands_num = dataframe['品牌'].nunique()
ws_rg_cG = [brands[br] for br in ws_rg_cF[:brands_num]]
ws_rg_cG += range_value

cH_string = """=CONCATENATE(A{}," ",D{}," ",E{}," ","="," ","'",G{},"'",";")"""
ws_rg_cH = [cH_string.format(i,i,i,i) for i in range(2, attr_num+2)]
cI_string = '=CONCATENATE("emxFramework.Range.",D{},,".",G{}," ="," ",F{})'
ws_rg_cI = [cI_string.format(i,i,i) for i in range(2, attr_num+2)]

# 将数据传入df
df = pd.DataFrame(columns=ws_rg_header)
df['固定属性'] = ws_rg_cA
df['分类码'] = ws_rg_cB
df['英文名字'] = ws_rg_cC
df['定义属性'] = ws_rg_cD
df['-'] = ws_rg_cE
df['值的前台显示'] = ws_rg_cF
df['值(不能有中文)'] = ws_rg_cG
df['Range值脚本'] = ws_rg_cH
df['Range值翻译'] = ws_rg_cI

# 将df写入Excel
for r in dataframe_to_rows(df, index=False, header=True):
    ws_rg.append(r)
wb.save("t01.xlsx")
wb.close()

## pipline